<a href="https://colab.research.google.com/github/Beatriz-Yun/AIFFEL_LMS/blob/main/Exploration/%5BE_02%5D%EB%B6%84%EB%A5%98%EB%AA%A8%EB%8D%B8_%ED%95%99%EC%8A%B5_%EB%B0%8F_%ED%8F%89%EA%B0%80.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 데이터 살펴보기

In [ ]:
from sklearn.datasets import load_digits
from sklearn.datasets import load_wine
from sklearn.datasets import load_breast_cancer

# feature data 지정
digits = load_digits()
wine = load_wine()
cancer = load_breast_cancer()

print(dir(digits))
print(dir(wine))
print(dir(cancer))

['DESCR', 'data', 'images', 'target', 'target_names']
['DESCR', 'data', 'feature_names', 'target', 'target_names']
['DESCR', 'data', 'feature_names', 'filename', 'target', 'target_names']


In [ ]:
# 데이터는 총 1797개, feature는 64개의 픽셀값 (8x8 이미지), label은 0~9 사이의 숫자
print(digits.data.shape, digits.target_names)
print(digits.target.shape)
print()

# 데이터는 총 178개, feature는 총 13개로, Alcohol, Malic acid, Color intensity 등 와인의 특성값이고,
# label은 class 0, 1, 2의 세 가지 카테고리가 있다.
print(wine.data.shape, wine.feature_names)
print(wine.target.shape, wine.target_names)
print()

# 데이터는 총 569개, feature는 총 30개로, 'mean radius' 'mean texture' 'mean perimeter' 등 유방암의 특성값이고,
# label은 malignant(악성)와 benign(양성) 2가지이다.
print(cancer.data.shape, cancer.feature_names)
print(len(cancer.feature_names))
print(cancer.target.shape, cancer.target_names)

(1797, 64) [0 1 2 3 4 5 6 7 8 9]
(1797,)

(178, 13) ['alcohol', 'malic_acid', 'ash', 'alcalinity_of_ash', 'magnesium', 'total_phenols', 'flavanoids', 'nonflavanoid_phenols', 'proanthocyanins', 'color_intensity', 'hue', 'od280/od315_of_diluted_wines', 'proline']
(178,) ['class_0' 'class_1' 'class_2']

(569, 30) ['mean radius' 'mean texture' 'mean perimeter' 'mean area'
 'mean smoothness' 'mean compactness' 'mean concavity'
 'mean concave points' 'mean symmetry' 'mean fractal dimension'
 'radius error' 'texture error' 'perimeter error' 'area error'
 'smoothness error' 'compactness error' 'concavity error'
 'concave points error' 'symmetry error' 'fractal dimension error'
 'worst radius' 'worst texture' 'worst perimeter' 'worst area'
 'worst smoothness' 'worst compactness' 'worst concavity'
 'worst concave points' 'worst symmetry' 'worst fractal dimension']
30
(569,) ['malignant' 'benign']


## 실습1. 손글씨 분류

In [ ]:
print(digits.DESCR)

.. _digits_dataset:

Optical recognition of handwritten digits dataset
--------------------------------------------------

**Data Set Characteristics:**

    :Number of Instances: 5620
    :Number of Attributes: 64
    :Attribute Information: 8x8 image of integer pixels in the range 0..16.
    :Missing Attribute Values: None
    :Creator: E. Alpaydin (alpaydin '@' boun.edu.tr)
    :Date: July; 1998

This is a copy of the test set of the UCI ML hand-written digits datasets
https://archive.ics.uci.edu/ml/datasets/Optical+Recognition+of+Handwritten+Digits

The data set contains images of hand-written digits: 10 classes where
each class refers to a digit.

Preprocessing programs made available by NIST were used to extract
normalized bitmaps of handwritten digits from a preprinted form. From a
total of 43 people, 30 contributed to the training set and different 13
to the test set. 32x32 bitmaps are divided into nonoverlapping blocks of
4x4 and the number of on pixels are counted in each blo

In [ ]:
# label data 지정
digits_label = digits.target
print(digits.target)
digits.target.shape

[0 1 2 ... 8 9 8]


(1797,)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# 데이터셋 분리 (8:2비율)
X_train, X_test, y_train, y_test = train_test_split(digits.data, digits_label, test_size=0.2, random_state=15)

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(1437, 64) (1437,)
(360, 64) (360,)


### 오차행렬 (confusion matrix)
- 예측결과: TN(True Negative), FP(False Positive), FN(False Negative), TP(True Positive)
- 성능 지표: Precision, Negative Predictive Value, Sensitivity, Specificity(Recall), Accuracy

> TP는 **사실과 예측결과가 Positive-Positive**여서 제대로 예측했으므로 True<br>
FP는 사실은 Negative인데 **예측결과가 Positive**여서 잘못 예측했으므로 False<br>
TN은 **사실과 예측결과 모두 Negative-Negative**여서 제대로 예측했으므로 True<br>
FN는 사실은 Positive인데 **예측결과가 Negative**여서 잘못 예측했으므로 False

<br>

> Precision(정밀도): Positive라고 예측한 데이터(FP+TP) 중 진짜 Positive(TP)인 비율<br>
Recall: 실제 Positive인 데이터(TP+FN) 중 Positive라고 예측 성공(TP)한 비율<br>

예시)
- 정상 메일을 스팸으로 분류하면 안된다.
= Negative를 Positive로 판단(FP)하면 안된다.
= precision이 중요하다.
- 암인데 정상이라고 판단하면 안된다.
= Positive인데 Negative라고 예측(FN)하면 안된다.
= FN이 중요하다.
= recall이 중요하다.

[참고](https://manisha-sirsat.blogspot.com/2019/04/confusion-matrix.html)

### 1-1. Decision Tree

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix
import numpy as np

decision_tree = DecisionTreeClassifier(random_state=32)    # 모델 로드

decision_tree.fit(X_train, y_train)                        # 모델 학습

y_pred = decision_tree.predict(X_test)                     # 모델 예측(테스트)

print(classification_report(y_test, y_pred))               # 성능 평가

cm = confusion_matrix(y_test, y_pred)                      # 왼쪽 위부터 순서대로 TP, FN, FP, TN의 개수를 나타낸다.
cm.sum() - np.diag(cm).sum()                               # 오차행렬의 총합에서 대각에 있는 값들을 제외 ( = False인 총 경우의 수)

              precision    recall  f1-score   support

           0       1.00      0.94      0.97        31
           1       0.82      0.82      0.82        38
           2       0.72      0.87      0.79        38
           3       0.85      0.81      0.83        27
           4       0.97      0.78      0.86        41
           5       0.82      0.89      0.85        35
           6       0.85      0.89      0.87        38
           7       0.91      0.91      0.91        34
           8       0.74      0.74      0.74        35
           9       0.83      0.79      0.81        43

    accuracy                           0.84       360
   macro avg       0.85      0.84      0.84       360
weighted avg       0.85      0.84      0.84       360



57

### 1-2. Random forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

random_forest = RandomForestClassifier(random_state=32)    # 모델 로드

random_forest.fit(X_train, y_train)                        # 모델 학습

y_pred = random_forest.predict(X_test)                     # 모델 예측(테스트)

print(classification_report(y_test, y_pred))               # 성능 평가

cm = confusion_matrix(y_test, y_pred)                      # 오차행렬
cm.sum() - np.diag(cm).sum()                               # False인 총 경우의 수

              precision    recall  f1-score   support

           0       0.97      0.94      0.95        31
           1       0.95      0.97      0.96        38
           2       1.00      1.00      1.00        38
           3       1.00      0.96      0.98        27
           4       0.95      1.00      0.98        41
           5       0.97      1.00      0.99        35
           6       1.00      0.95      0.97        38
           7       1.00      1.00      1.00        34
           8       0.94      0.97      0.96        35
           9       1.00      0.98      0.99        43

    accuracy                           0.98       360
   macro avg       0.98      0.98      0.98       360
weighted avg       0.98      0.98      0.98       360



8

### 1-3. SVM

In [ ]:
from sklearn import svm

svm_model = svm.SVC()    # SVM모델 로드

svm_model.fit(X_train, y_train)               # SVM모델 학습시키기

y_pred = svm_model.predict(X_test)            # 모델 예측(테스트)

print(classification_report(y_test, y_pred))  # 모델 성능 평가

cm = confusion_matrix(y_test, y_pred)         # 오차행렬
cm.sum() - np.diag(cm).sum()                  # False인 총 경우의 수

              precision    recall  f1-score   support

           0       1.00      0.97      0.98        31
           1       0.95      1.00      0.97        38
           2       1.00      1.00      1.00        38
           3       0.96      0.96      0.96        27
           4       0.98      0.98      0.98        41
           5       1.00      1.00      1.00        35
           6       1.00      1.00      1.00        38
           7       1.00      1.00      1.00        34
           8       0.97      0.94      0.96        35
           9       0.98      0.98      0.98        43

    accuracy                           0.98       360
   macro avg       0.98      0.98      0.98       360
weighted avg       0.98      0.98      0.98       360



6

### 1-4. SGD Classifier

In [ ]:
from sklearn.linear_model import SGDClassifier

sgd_model = SGDClassifier()    # 모델 로드

sgd_model.fit(X_train, y_train)                # 모델 학습

y_pred = sgd_model.predict(X_test)             # 모델 예측(테스트)

print(classification_report(y_test, y_pred))   # 모델 성능 평가

cm = confusion_matrix(y_test, y_pred)          # 오차행렬
cm.sum() - np.diag(cm).sum()                   # False인 총 경우의 수

              precision    recall  f1-score   support

           0       1.00      0.97      0.98        31
           1       0.97      0.89      0.93        38
           2       0.97      0.95      0.96        38
           3       0.96      0.85      0.90        27
           4       0.95      0.98      0.96        41
           5       0.97      0.94      0.96        35
           6       1.00      0.95      0.97        38
           7       0.97      1.00      0.99        34
           8       0.80      0.91      0.85        35
           9       0.87      0.95      0.91        43

    accuracy                           0.94       360
   macro avg       0.95      0.94      0.94       360
weighted avg       0.95      0.94      0.94       360



21

### 1-5. Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression

logistic_model = LogisticRegression()         # 모델 로드

logistic_model.fit(X_train, y_train)          # 모델 학습

y_pred = logistic_model.predict(X_test)       # 모델 예측(테스트)

print(classification_report(y_test, y_pred))  # 모델 성능 평가

cm = confusion_matrix(y_test, y_pred)         # 오차행렬
cm.sum() - np.diag(cm).sum()                  # False인 총 경우의 수

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        31
           1       0.95      0.97      0.96        38
           2       1.00      0.97      0.99        38
           3       0.96      0.93      0.94        27
           4       0.98      1.00      0.99        41
           5       0.97      0.97      0.97        35
           6       1.00      0.97      0.99        38
           7       1.00      1.00      1.00        34
           8       0.92      0.97      0.94        35
           9       0.98      0.95      0.96        43

    accuracy                           0.97       360
   macro avg       0.98      0.97      0.97       360
weighted avg       0.98      0.97      0.98       360



/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


9

### 손글씨 분류 모델 성능 평가

손글씨 분류에서 중요한 것은 잘못 인식하는 것을 줄이는 것이다.
그래서 FN과 FP인 경우(False)가 제일 적은 모델을 선택한다.

![이미지](https://drive.google.com/uc?export=view&id=1J9cvlvhc_AFebfq_K7KKY86l7MPJK-Ls)

skelarn.metrics를 사용하여 confusion matrix을 생성하면
**confusion matrix의 대각은 True**인 경우이므로,<br>
행렬의 총합에서 대각을 제외하면 **False인 총 경우의 수를 서로 비교**할 수 있다.

decision tree, random forest, SVM, SGD Classifier, Logistic Regression에서 예측결과가 False인 경우는 각각 57, 8, 6, 21, 9이다.<br>
따라서 손글씨 분류모델은 **SVM모델**을 사용하였을 때 성능이 제일 좋다.

## 실습2. 와인 분류

In [ ]:
print(wine.DESCR)

.. _wine_dataset:

Wine recognition dataset
------------------------

**Data Set Characteristics:**

    :Number of Instances: 178 (50 in each of three classes)
    :Number of Attributes: 13 numeric, predictive attributes and the class
    :Attribute Information:
 		- Alcohol
 		- Malic acid
 		- Ash
		- Alcalinity of ash  
 		- Magnesium
		- Total phenols
 		- Flavanoids
 		- Nonflavanoid phenols
 		- Proanthocyanins
		- Color intensity
 		- Hue
 		- OD280/OD315 of diluted wines
 		- Proline

    - class:
            - class_0
            - class_1
            - class_2
		
    :Summary Statistics:
    
    ============================= ==== ===== ======= =====
                                   Min   Max   Mean     SD
    ============================= ==== ===== ======= =====
    Alcohol:                      11.0  14.8    13.0   0.8
    Malic Acid:                   0.74  5.80    2.34  1.12
    Ash:                          1.36  3.23    2.36  0.27
    Alcalinity of Ash:            1

In [ ]:
# label data 지정
wine_label = wine.target

print(wine_label)
print(wine_label.shape)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2]
(178,)


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report


# 데이터셋 분리
X_train, X_test, y_train, y_test = train_test_split(wine.data, wine_label, test_size=0.2, random_state=15)

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(142, 13) (142,)
(36, 13) (36,)


### 2-1. Decision Tree

In [ ]:
from sklearn.tree import DecisionTreeClassifier

decision_tree = DecisionTreeClassifier(random_state=32)    # 모델 로드

decision_tree.fit(X_train, y_train)                        # 모델 학습

y_pred = decision_tree.predict(X_test)                     # 모델 예측(테스트)

print(classification_report(y_test, y_pred))               # 성능 평가

cm = confusion_matrix(y_test, y_pred)         # 오차행렬
cm.sum() - np.diag(cm).sum()                  # False인 총 경우의 수

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        12
           1       0.85      0.92      0.88        12
           2       0.91      0.83      0.87        12

    accuracy                           0.92        36
   macro avg       0.92      0.92      0.92        36
weighted avg       0.92      0.92      0.92        36



3

### 2-2. Random forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

random_forest = RandomForestClassifier(random_state=32)    # 모델 로드

random_forest.fit(X_train, y_train)                        # 모델 학습

y_pred = random_forest.predict(X_test)                     # 모델 예측(테스트)

print(classification_report(y_test, y_pred))               # 성능 평가

cm = confusion_matrix(y_test, y_pred)         # 오차행렬
cm.sum() - np.diag(cm).sum()                  # False인 총 경우의 수

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        12
           1       1.00      1.00      1.00        12
           2       1.00      1.00      1.00        12

    accuracy                           1.00        36
   macro avg       1.00      1.00      1.00        36
weighted avg       1.00      1.00      1.00        36



0

### 2-3. SVM

In [ ]:
from sklearn import svm

svm_model = svm.SVC()    # SVM모델 로드

svm_model.fit(X_train, y_train)               # SVM모델 학습시키기

y_pred = svm_model.predict(X_test)            # 모델 예측(테스트)

print(classification_report(y_test, y_pred))  # 모델 성능 평가

cm = confusion_matrix(y_test, y_pred)         # 오차행렬
cm.sum() - np.diag(cm).sum()                  # False인 총 경우의 수

              precision    recall  f1-score   support

           0       0.79      0.92      0.85        12
           1       0.50      0.92      0.65        12
           2       0.00      0.00      0.00        12

    accuracy                           0.61        36
   macro avg       0.43      0.61      0.50        36
weighted avg       0.43      0.61      0.50        36



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


14

### 2-4. SGD Classifier

In [ ]:
from sklearn.linear_model import SGDClassifier

sgd_model = SGDClassifier()    # 모델 로드

sgd_model.fit(X_train, y_train)                # 모델 학습

y_pred = sgd_model.predict(X_test)             # 모델 예측(테스트)

print(classification_report(y_test, y_pred))   # 모델 성능 평가

cm = confusion_matrix(y_test, y_pred)         # 오차행렬
cm.sum() - np.diag(cm).sum()                  # False인 총 경우의 수

              precision    recall  f1-score   support

           0       0.77      0.83      0.80        12
           1       0.43      0.83      0.57        12
           2       0.00      0.00      0.00        12

    accuracy                           0.56        36
   macro avg       0.40      0.56      0.46        36
weighted avg       0.40      0.56      0.46        36



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


16

### 2-5. Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression

logistic_model = LogisticRegression()         # 모델 로드

logistic_model.fit(X_train, y_train)          # 모델 학습

y_pred = logistic_model.predict(X_test)       # 모델 예측(테스트)

print(classification_report(y_test, y_pred))  # 모델 성능 평가

cm = confusion_matrix(y_test, y_pred)         # 오차행렬
cm.sum() - np.diag(cm).sum()                  # False인 총 경우의 수

              precision    recall  f1-score   support

           0       1.00      0.92      0.96        12
           1       0.86      1.00      0.92        12
           2       1.00      0.92      0.96        12

    accuracy                           0.94        36
   macro avg       0.95      0.94      0.95        36
weighted avg       0.95      0.94      0.95        36



/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


2

학습된 모델들의 테스트데이터 예측 결과를 어떻게 해석해야 할까요? 모델의 성능을 평가하는 지표로는 무엇이 좋을까요? sklearn.metrics 에서 제공하는 평가지표 중 적절한 것을 선택해 보세요. 선택하신 이유도 설명해 주세요.


### 와인 분류 모델 성능 평가

와인 분류에서 중요한 것은 False인 경우를 줄이는 것이라고 생각했다.<br>
그래서 confusion matrix의 요소를 직접 더하며 계산해보았을 때 False가 0이었던, 즉 정확도가 100%인 Random Forest모델이 성능이 제일 좋다.

## 실습3. 유방암 여부 진단

In [ ]:
print(cancer.DESCR)

.. _breast_cancer_dataset:

Breast cancer wisconsin (diagnostic) dataset
--------------------------------------------

**Data Set Characteristics:**

    :Number of Instances: 569

    :Number of Attributes: 30 numeric, predictive attributes and the class

    :Attribute Information:
        - radius (mean of distances from center to points on the perimeter)
        - texture (standard deviation of gray-scale values)
        - perimeter
        - area
        - smoothness (local variation in radius lengths)
        - compactness (perimeter^2 / area - 1.0)
        - concavity (severity of concave portions of the contour)
        - concave points (number of concave portions of the contour)
        - symmetry 
        - fractal dimension ("coastline approximation" - 1)

        The mean, standard error, and "worst" or largest (mean of the three
        largest values) of these features were computed for each image,
        resulting in 30 features.  For instance, field 3 is Mean Radius, f

In [ ]:
# label data 지정
cancer_label = cancer.target

print(cancer_label)
print(cancer_label.shape)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 1 0 0 0 0 0 0 0 0 1 0 1 1 1 1 1 0 0 1 0 0 1 1 1 1 0 1 0 0 1 1 1 1 0 1 0 0
 1 0 1 0 0 1 1 1 0 0 1 0 0 0 1 1 1 0 1 1 0 0 1 1 1 0 0 1 1 1 1 0 1 1 0 1 1
 1 1 1 1 1 1 0 0 0 1 0 0 1 1 1 0 0 1 0 1 0 0 1 0 0 1 1 0 1 1 0 1 1 1 1 0 1
 1 1 1 1 1 1 1 1 0 1 1 1 1 0 0 1 0 1 1 0 0 1 1 0 0 1 1 1 1 0 1 1 0 0 0 1 0
 1 0 1 1 1 0 1 1 0 0 1 0 0 0 0 1 0 0 0 1 0 1 0 1 1 0 1 0 0 0 0 1 1 0 0 1 1
 1 0 1 1 1 1 1 0 0 1 1 0 1 1 0 0 1 0 1 1 1 1 0 1 1 1 1 1 0 1 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 1 1 1 1 1 1 0 1 0 1 1 0 1 1 0 1 0 0 1 1 1 1 1 1 1 1 1 1 1 1
 1 0 1 1 0 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 0 1 0 1 1 1 1 0 0 0 1 1
 1 1 0 1 0 1 0 1 1 1 0 1 1 1 1 1 1 1 0 0 0 1 1 1 1 1 1 1 1 1 1 1 0 0 1 0 0
 0 1 0 0 1 1 1 1 1 0 1 1 1 1 1 0 1 1 1 0 1 1 0 0 1 1 1 1 1 1 0 1 1 1 1 1 1
 1 0 1 1 1 1 1 0 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 0 1 0 0 1 0 1 1 1 1 1 0 1 1
 0 1 0 1 1 0 1 0 1 1 1 1 1 1 1 1 0 0 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 0 1
 1 1 1 1 1 1 0 1 0 1 1 0 

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report


# 데이터셋 분리
X_train, X_test, y_train, y_test = train_test_split(cancer.data, cancer_label, test_size=0.2, random_state=15)

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(455, 30) (455,)
(114, 30) (114,)


### 3-1. Decision Tree

In [ ]:
from sklearn.tree import DecisionTreeClassifier

decision_tree = DecisionTreeClassifier(random_state=32)    # 모델 로드

decision_tree.fit(X_train, y_train)                        # 모델 학습

y_pred = decision_tree.predict(X_test)                     # 모델 예측(테스트)

print(classification_report(y_test, y_pred))               # 성능 평가

              precision    recall  f1-score   support

           0       0.97      0.87      0.92        39
           1       0.94      0.99      0.96        75

    accuracy                           0.95       114
   macro avg       0.95      0.93      0.94       114
weighted avg       0.95      0.95      0.95       114



### 3-2. Random forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

random_forest = RandomForestClassifier(random_state=32)    # 모델 로드

random_forest.fit(X_train, y_train)                        # 모델 학습

y_pred = random_forest.predict(X_test)                     # 모델 예측(테스트)

print(classification_report(y_test, y_pred))               # 성능 평가

              precision    recall  f1-score   support

           0       0.94      0.87      0.91        39
           1       0.94      0.97      0.95        75

    accuracy                           0.94       114
   macro avg       0.94      0.92      0.93       114
weighted avg       0.94      0.94      0.94       114



### 3-3. SVM

In [ ]:
from sklearn import svm

svm_model = svm.SVC()    # SVM모델 로드

svm_model.fit(X_train, y_train)               # SVM모델 학습시키기

y_pred = svm_model.predict(X_test)            # 모델 예측(테스트)

print(classification_report(y_test, y_pred))  # 모델 성능 평가

              precision    recall  f1-score   support

           0       0.93      0.67      0.78        39
           1       0.85      0.97      0.91        75

    accuracy                           0.87       114
   macro avg       0.89      0.82      0.84       114
weighted avg       0.88      0.87      0.86       114



### 3-4. SGD Classifier

In [ ]:
from sklearn.linear_model import SGDClassifier

sgd_model = SGDClassifier()    # 모델 로드

sgd_model.fit(X_train, y_train)                # 모델 학습

y_pred = sgd_model.predict(X_test)             # 모델 예측(테스트)

print(classification_report(y_test, y_pred))   # 모델 성능 평가

              precision    recall  f1-score   support

           0       1.00      0.36      0.53        39
           1       0.75      1.00      0.86        75

    accuracy                           0.78       114
   macro avg       0.88      0.68      0.69       114
weighted avg       0.84      0.78      0.74       114



### 3-5. Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression

logistic_model = LogisticRegression()         # 모델 로드

logistic_model.fit(X_train, y_train)          # 모델 학습

y_pred = logistic_model.predict(X_test)       # 모델 예측(테스트)

print(classification_report(y_test, y_pred))  # 모델 성능 평가

              precision    recall  f1-score   support

           0       0.85      0.85      0.85        39
           1       0.92      0.92      0.92        75

    accuracy                           0.89       114
   macro avg       0.88      0.88      0.88       114
weighted avg       0.89      0.89      0.89       114



/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


학습된 모델들의 테스트데이터 예측 결과를 어떻게 해석해야 할까요? 모델의 성능을 평가하는 지표로는 무엇이 좋을까요? sklearn.metrics 에서 제공하는 평가지표 중 적절한 것을 선택해 보세요. 선택하신 이유도 설명해 주세요.


### 유방암 여부 진단 모델 성능 평가

유방암 진단에서 중요한 것은 유방암 환자를 정상인으로 판단하는 오류(FP)를 줄이는 것이다.<br>
FP가 작을수록 커지는 precision척도를 사용해서 모델의 성능을 비교해보자.<br>

decision tree, random forest, SVM, SGD Classifier, Logistic Regression  중 precision의 avg값이 제일 큰 **decision tree 모델**을 선택하는 것이 좋다.

이번 기회에 decision tree, SVM, random forest에 대해 어느정도 이해하게 되었고, 데이터의 특성마다 평가방식을 달리해야한다는 것과 각 데이터마다 성능이 좋은 모델이 다른 것이 흥미로웠다. 단지 sklearn에서 제공하는 모델을 학습시킬 때 내부적으로 어떻게 동작하는 지 더 알아보고 싶다.